#### Hive Internal and External Tables

#### Create File For Demonstration

In [0]:
dbutils.fs.put("/FileStore/students.csv","""id,Name
1,john
2,dane
3,stallone
4,slyvester
5,steve
6,tony
7,bruce""",overwrite=True)

Wrote 67 bytes.
Out[36]: True

In [0]:
%fs head /FileStore/students.csv

id,Name
1,john
2,dane
3,stallone
4,slyvester
5,steve
6,tony
7,bruce

##### Managed/Hive Tables
- You can check the filesystem with %fs ls user/hive/warehouse/
- you can remove the files with %fs rm -r user/hive/warehouse/sharmatrilok.db

In [0]:
%sql
CREATE TABLE student(
id INT,
Name STRING)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ',';

In [0]:
%sql
describe extended student;

col_name,data_type,comment
id,int,null
Name,string,null
,,
# Detailed Table Information,,
Database,default,
Table,student,
Owner,root,
Created Time,Mon Dec 05 08:16:32 UTC 2022,
Last Access,UNKNOWN,
Created By,Spark 3.2.1,


In [0]:
%sql
LOAD DATA LOCAL INPATH 'dbfs:/FileStore/students.csv' INTO TABLE student;

In [0]:
%sql
select * from student;

id,Name
null,Name
1,john
2,dane
3,stallone
4,slyvester
5,steve
6,tony
7,bruce


In [0]:
%sql
truncate table student;

In [0]:
%sql
select * from student;

id,Name


In [0]:
%sql
describe extended student;

col_name,data_type,comment
id,int,null
Name,string,null
,,
# Detailed Table Information,,
Database,default,
Table,student,
Owner,root,
Created Time,Mon Dec 05 08:16:32 UTC 2022,
Last Access,UNKNOWN,
Created By,Spark 3.2.1,


In [0]:
%sql
drop table student;

#### External Tables

In [0]:
dbutils.fs.mkdirs("/FileStore/Students")

In [0]:
dbutils.fs.cp("/FileStore/students.csv","/FileStore/Students/")

In [0]:
%sql
CREATE EXTERNAL TABLE  IF NOT EXISTS student(
id INT,
Name STRING)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LOCATION 'dbfs:/FileStore/Students';

In [0]:
%sql
-- drop table if exists student;
describe extended student;

col_name,data_type,comment
id,int,null
Name,string,null
,,
# Detailed Table Information,,
Database,default,
Table,student,
Owner,root,
Created Time,Mon Dec 05 09:13:51 UTC 2022,
Last Access,UNKNOWN,
Created By,Spark 3.2.1,


In [0]:
%sql
SELECT * FROM student;

id,Name
null,Name
1,john
2,dane
3,stallone
4,slyvester
5,steve
6,tony
7,bruce
null,null
101,johnson


In [0]:
dbutils.fs.put("/FileStore/Students/students1.csv","""
101,johnson
102,quil
103,groot
""",overwrite=True)

Wrote 32 bytes.
Out[61]: True

In [0]:
%sql
SELECT * FROM student;

id,Name
null,Name
1,john
2,dane
3,stallone
4,slyvester
5,steve
6,tony
7,bruce
null,null
101,johnson


In [0]:
%sql
drop table student;

In [0]:
%sql
select * from student;

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-957540864188454> in <module>
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

<command-957540864188454> in ____databricks_percent_sql()
      2   def ____databricks_percent_sql():
      3     import base64
----> 4     df = spark.sql(base64.standard_b64decode("c2VsZWN0ICogZnJvbSBzdHVkZW50").decode())
      5     display(df)
      6     return df

/databricks/spark/python/pyspark/sql/session.py in sql(self, sqlQuery)
    775         [Row(f1=1, f2='row1'), Row(f1=2, f2='row2'), Row(f1=3, f2='row3')]
    776         """
--> 777         return DataFrame(self._jsparkSession.sql(sqlQuery), self._wrapped)
    778 
    779     def table(self, tableName):

/databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __cal

### Spark/Hive Partitions

##### To set Any Configuration in spark session we can use
```python
spark.conf.set("spark.sql.shuffle.partitions", "500")
./bin/spark-submit --conf spark.sql.shuffle.partitions=500 --conf spark.default.parallelism=500

```

#### Create Database

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS sharmatrilok;
USE sharmatrilok;

In [0]:
%sql
drop table if exists sales; 
CREATE TABLE sales(
 InvoiceNO          int         comment 'Invoice No',
 StockCode          int         comment 'Stock Code means product Id', 
 Description        string      comment 'Description',
 Quantity           int         comment 'Quantity of a product ordered in invoice',
 InvoiceDate        timestamp   comment 'Invoice Date',
 UnitPrice          double      comment 'Unit Price',
 CustomerID         int         comment 'Customer ID'
)
PARTITIONED BY (Country string)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
LINES TERMINATED BY '\n'
STORED AS TEXTFILE
LOCATION '/user/hive/warehouse/sharmatrilok';

In [0]:
spark.conf.set("hive.exec.dynamic.partition", "true")
spark.conf.set("hive.exec.dynamic.partition.mode", "nonstrict")

### Parsed Sales DATA according to Hive Table
- Spark support 'yyyy-MM-dd HH:MM:SS' format as timestamp.
- Need to convert the data into this format.

In [0]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,FloatType
from pyspark.sql.functions import col,to_date,lit,to_timestamp

In [0]:
schema = StructType(fields=[
    StructField("InvoiceNo",IntegerType(),nullable=False),
    StructField("StockCode",IntegerType(),nullable=True),
    StructField("Description",StringType(),nullable=True),
    StructField("Quantity",IntegerType(),nullable=True),
    StructField("InvoiceDate",StringType(),nullable=True),
    StructField("UnitPrice",FloatType(),nullable=True),
    StructField("CustomerID",IntegerType(),nullable=False),
    StructField("Country",StringType(),nullable=False)
])

In [0]:
df = spark.read\
    .format("csv")\
    .schema(schema)\
    .option("header",True)\
    .load("dbfs:/FileStore/sales.csv")

In [0]:
df.show(2,truncate=False)

+---------+---------+----------------------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                       |Quantity|InvoiceDate    |UnitPrice|CustomerID|Country       |
+---------+---------+----------------------------------+--------+---------------+---------+----------+--------------+
|536365   |null     |WHITE HANGING HEART T-LIGHT HOLDER|6       |01-12-2010 8.26|2.55     |17850     |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN               |6       |01-12-2010 8.26|3.39     |17850     |United Kingdom|
+---------+---------+----------------------------------+--------+---------------+---------+----------+--------------+
only showing top 2 rows



In [0]:
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")
from pyspark.sql.functions import split,regexp_replace,concat

In [0]:
df = df.withColumn("InvoiceDate",to_timestamp(col("InvoiceDate"),format="dd-MM-yyyy HH.mm"))

In [0]:
display(df)

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
536365,null,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01T08:26:00.000+0000,2.55,17850,United Kingdom
536365,71053,WHITE METAL LANTERN,6,2010-12-01T08:26:00.000+0000,3.39,17850,United Kingdom
536365,null,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01T08:26:00.000+0000,2.75,17850,United Kingdom
536365,null,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01T08:26:00.000+0000,3.39,17850,United Kingdom
536365,null,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01T08:26:00.000+0000,3.39,17850,United Kingdom
536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01T08:26:00.000+0000,7.65,17850,United Kingdom
536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01T08:26:00.000+0000,4.25,17850,United Kingdom
536366,22633,HAND WARMER UNION JACK,6,2010-12-01T08:28:00.000+0000,1.85,17850,United Kingdom
536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01T08:28:00.000+0000,1.85,17850,United Kingdom
536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01T08:34:00.000+0000,1.69,13047,United Kingdom


In [0]:
df.createOrReplaceTempView("SalesTemp")

In [0]:
%sql
insert into sales partition(country) select * from SalesTemp;

#### Read the data from partition

In [0]:
%sql
select * from sales where country="France";

InvoiceNO,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
536370,22728,ALARM CLOCK BAKELIKE PINK,24,2010-12-01T08:45:00.000+0000,3.75,12583,France
536370,22727,ALARM CLOCK BAKELIKE RED,24,2010-12-01T08:45:00.000+0000,3.75,12583,France
536370,22726,ALARM CLOCK BAKELIKE GREEN,12,2010-12-01T08:45:00.000+0000,3.75,12583,France
536370,21724,PANDA AND BUNNIES STICKER SHEET,12,2010-12-01T08:45:00.000+0000,0.8500000238418579,12583,France
536370,21883,STARS GIFT TAPE,24,2010-12-01T08:45:00.000+0000,0.6499999761581421,12583,France
536370,10002,INFLATABLE POLITICAL GLOBE,48,2010-12-01T08:45:00.000+0000,0.8500000238418579,12583,France
536370,21791,VINTAGE HEADS AND TAILS CARD GAME,24,2010-12-01T08:45:00.000+0000,1.25,12583,France
536370,21035,SET/2 RED RETROSPOT TEA TOWELS,18,2010-12-01T08:45:00.000+0000,2.950000047683716,12583,France
536370,22326,ROUND SNACK BOXES SET OF4 WOODLAND,24,2010-12-01T08:45:00.000+0000,2.950000047683716,12583,France
536370,22629,SPACEBOY LUNCH BOX,24,2010-12-01T08:45:00.000+0000,1.9500000476837158,12583,France


#### Bucketing in Hive

In [0]:
%sql
drop table if exists sales_bucket; 
CREATE TABLE sales_bucket(
 InvoiceNO          int         comment 'Invoice No',
 StockCode          int         comment 'Stock Code means product Id', 
 Description        string      comment 'Description',
 Quantity           int         comment 'Quantity of a product ordered in invoice',
 InvoiceDate        timestamp   comment 'Invoice Date',
 UnitPrice          double      comment 'Unit Price',
 CustomerID         int         comment 'Customer ID'
)
PARTITIONED BY (Country string)
CLUSTERED BY(CustomerID) INTO 5 BUCKETS
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
LINES TERMINATED BY '\n'
STORED AS TEXTFILE
LOCATION '/user/hive/warehouse/sharmatrilok.db/src';

In [0]:
%sql
insert into sales_bucket partition(Country)  Select * from SalesTemp;

In [0]:
%sql
describe extended sales_bucket;

col_name,data_type,comment
InvoiceNO,int,Invoice No
StockCode,int,Stock Code means product Id
Description,string,Description
Quantity,int,Quantity of a product ordered in invoice
InvoiceDate,timestamp,Invoice Date
UnitPrice,double,Unit Price
CustomerID,int,Customer ID
Country,string,null
# Partition Information,,
# col_name,data_type,comment
